# **Desafio Vivencial - Trilha Cientista de dados**


# 1. Introdução 
Este trabalho constitui desafio vivencial da trilha de formação de cientista de dados na PETROBRAS. Foi escolhida a competição de Santander Customer Transaction Prediction. Essa é uma competição encerrada em Abril de 2019 que ofereceu US$ 65.000 do 1° ao 5° lugares. Nessa competição, o objetivo é prever quais clientes irão realizar uma transação específica no futuro, independente do valor.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df= pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
sample_submission = pd.read_csv('../input/santander-customer-transaction-prediction/sample_submission.csv')
sample_submission.head()

2. Análise exploratória de dados - EDA 
Referências utilizadas:

Santander EDA and Prediction - https://www.kaggle.com/code/gpreda/santander-eda-and-prediction/notebook

Pandas documentation

Na descrição dos dados. é informado que os valores disponibilizados para previsão são numéricos e anonimizados, (conforme observado em Santander Customer Transaction - EDA) e tem estrutura similar à disponível para a solução do probelma, o que sugere que os dados são sintéticos. Por se tratar de informação de transações financeiras, é compreensível que os dados sejam anonimizados, porém o fato de não saber o que cada uma das muitas colunas significa, dificulta a análise de dados e possíveis idéias de combinação de variáveis para melhorar o resultado do modelo.

In [ ]:
train_df.describe()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,6))
features = train_df.columns.values[2:202]
plt.title("Distribuição dos valores médios por coluna nos conjuntos de treino e de teste")
sns.distplot(train_df[features].mean(axis=1),color="green", kde=True,bins=120, label='train')
sns.distplot(test_df[features].mean(axis=1),color="blue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

In [ ]:
train_df.info(verbose = True, show_counts = True)

In [ ]:
sns.countplot(x = train_df.target.values)

In [ ]:
import matplotlib.pyplot as plt
def plot_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(18,44))

    for feature in features:
        i += 1
        plt.subplot(20,10,i)
        sns.kdeplot(df1[feature],
                    label=label1)
        sns.kdeplot(df2[feature],
                     label=label2)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[2:202]
plot_feature_distribution(t0, t1, '0', '1', features)

As informações a seguir foram adaptadas do notebook Mutual Information, que é parte do treinamento de Feature Engineering. Nesse desafio, existe uma grande quantidade de variáveis de entrada e não é fornecida informação adicional sobre elas. Nessa situação, se faz pertinente a criação de uma métrica de utilidade das variáveis, uma função associando as variáveis de entrada com a variável de saída. A partir dessa métrica, é possível escolher um subconjunto das variáveis de entrada mais relacionadas com a variável de saída. A métrica utilizada será a de Mutual Information, que se assemelha bastante à correlação com a diferença de que, enquanto a correlação detecta relações lineares, e Mutual Information pode detectar qualquer tipo de relação. Essa seleção de features também se mostrou útil na comparação de modelos. Ao tentar rodar a função pycaret.classification.compare_models() com todas as features, a imagem do Kaggle apresenta erro de falta de memória, entre outros.

In [ ]:
X = train_df.copy()
y = X.pop("target")
X.pop("ID_code")
discrete_features = X.dtypes == int
X.head()